<a href="https://colab.research.google.com/github/AndrewCDownie/Classwork/blob/master/Story_Cloze_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
import pprint
pp = pprint.PrettyPrinter()
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizer


In [3]:
import torch
cpu = torch.device("cpu")

if torch.cuda.is_available():
  print("GPU avalible")
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
  print("Using CPU")
torch.cuda.empty_cache()

GPU avalible


In [ ]:
import os
print(os.listdir())

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#load the data set
cloze_test_data = pd.read_csv("drive/My Drive/CS 680 Project/cloze_test_val__winter2018-cloze_test_ALL_val - 1 - 1.csv")

In [ ]:
#Data frame column names
sentence_cols = ["InputSentence1","InputSentence2","InputSentence3","InputSentence4"]
endings_cols = ["RandomFifthSentenceQuiz1","RandomFifthSentenceQuiz2"]
answer_col = "AnswerRightEnding"

#information needed for training
stories = []
endings = []
correct_endings = []

#iterate through rows collecting the stories, the ending and the correct answers
for _,row in cloze_test_data.iterrows():
  story = ""
  for col in sentence_cols:
    story += " "+row[col]
  
  endings.append([row[endings_cols[0]],row[endings_cols[1]]])
  stories.append(story)
  correct_endings.append(int(row[answer_col])-1)

In [4]:
class cloze_data_set(Dataset):
  def __init__(self,file_path):
    #open the file
    cloze_test_data = pd.read_csv(file_path)
    
    #column names
    sentence_cols = ["InputSentence1","InputSentence2","InputSentence3","InputSentence4"]
    endings_cols = ["RandomFifthSentenceQuiz1","RandomFifthSentenceQuiz2"]
    answer_col = "AnswerRightEnding"

    #Stories 
    self.stories = []
    self.endings = []
    self.correct_endings = []
    self.tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case = True)
    self.input_data = []
    self.labels = []
    #iterate through rows and get the data 
    for _,row in cloze_test_data.iterrows():
      story = ""
      for col in sentence_cols:
        story += " "+row[col]
      self.endings.append([row[endings_cols[0]],row[endings_cols[1]]])
      self.stories.append(story)
      self.correct_endings.append(int(row[answer_col])-1)

    #convert the data points to input acceptable data

    for i in range(len(self.stories)):#(len(self.stories)):
      data_point = [[self.stories[i],self.endings[i][0]],[self.stories[i],self.endings[i][1]]]
      label = torch.LongTensor([self.correct_endings[i]])
      encoding = self.tokenizer.batch_encode_plus(data_point,return_tensors="pt",pad_to_max_length=True,max_length = 128,truncation=True)
      data = {k: v.unsqueeze(0) for k,v in encoding.items()}
      self.input_data.append(data)
      self.labels.append(label)

  def __len__(self):
    return len(self.input_data)
    

  def __getitem__(self,idx):
    return{"input_data":self.input_data[idx],"label":self.labels[idx],"story":self.stories[idx],"endings":self.endings[idx]}


  def collate_function(self,batch):
    input_id_tensors = []
    input_ids_sets = []
    token_type_ids_sets = []
    attention_mask_sets = []
    labels = []
    for element in batch:
      input_ids_sets.append(element['input_data']['input_ids'][0])
      token_type_ids_sets.append(element['input_data']['token_type_ids'][0])
      attention_mask_sets.append(element['input_data']['attention_mask'][0])
      labels.append(element['label'][0].item())
    
      
    return {"input_data":{'input_ids':torch.stack(input_ids_sets),"token_type_ids":torch.stack(token_type_ids_sets),"attention_mask":torch.stack(attention_mask_sets)},'labels':torch.LongTensor(labels)}
    #for input_data in 


In [5]:
file_path = "drive/My Drive/CS 680 Project/cloze_test_val__winter2018-cloze_test_ALL_val - 1 - 1.csv"
data = cloze_data_set(file_path)
data_loader = DataLoader(data,batch_size = 1)
batch = data.collate_function([data[0],data[1],data[2]])
train_set,val_set = torch.utils.data.random_split(data,[int(0.9*len(data)),len(data)-int(0.9*len(data))])

print(train_set)
train_loader = DataLoader(train_set,batch_size = 32,collate_fn=data.collate_function)
from transformers import BertModel, BertConfig,BertForMultipleChoice
import torch
configuration = BertConfig()
MultiChoiceModel = BertForMultipleChoice.from_pretrained("bert-base-uncased")
MultiChoiceModel.cuda()
MultiChoiceModel.train()
for i,batch in enumerate(train_loader):
  input_ids = batch['input_data']['input_ids'].to(device)
  token_type_ids = batch['input_data']['token_type_ids'].to(device)
  attention_mask = batch['input_data']['attention_mask'].to(device)
  labels= batch['labels'].to(device)
  print(MultiChoiceModel(input_ids = input_ids,token_type_ids = token_type_ids,attention_mask = attention_mask, labels= labels))
  input_ids = batch['input_data']['input_ids'].to(cpu)
  token_type_ids = batch['input_data']['token_type_ids'].to(cpu)
  attention_mask = batch['input_data']['attention_mask'].to(cpu)
  labels= batch['labels'].to(cpu)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

(tensor(0.7219, device='cuda:0', grad_fn=<NllLossBackward>), tensor([[-0.7337, -1.1184],
        [-0.7440, -0.7446],
        [-0.9676,  0.5069],
        [-0.6957, -1.0350],
        [-0.4504, -0.9006],
        [-0.6906, -1.1891],
        [-0.8168, -0.9468],
        [-0.8777, -0.5554],
        [-0.8857, -0.9517],
        [-0.9989, -1.0307],
        [-1.2596, -0.7607],
        [-0.6441, -0.9149],
        [-0.9198,  0.0392],
        [-0.8704, -0.5094],
        [-1.1352, -0.7142],
        [-0.8450, -0.6842],
        [-0.8473, -0.8052],
        [-0.9431, -0.7378],
        [-0.9256, -0.9979],
        [-1.0153, -1.0378],
        [-0.6235, -0.7130],
        [-0.9762, -1.0211],
        [-0.7841, -1.0883],
        [-0.7748, -0.5995],
        [-0.7928, -0.8448],
        [-0.9151, -0.8666],
        [-0.7786, -0.7550],
        [-1.0030, -0.7964],
        [-0.4323, -0.4950],
        [-0.8153, -1.1051],
        [-0.9235, -0.8909],
        [-0.7642, -0.7276]], device='cuda:0', grad_fn=<ViewBackward>))


In [ ]:
from transformers import BertModel, BertConfig,BertForMultipleChoice
import torch
configuration = BertConfig()
MultiChoiceModel = BertForMultipleChoice.from_pretrained("bert-base-uncased")


In [ ]:
!pip install transformers

     |████████████████████████████████| 675kB 6.0MB/s 
     |████████████████████████████████| 3.8MB 12.2MB/s 
     |████████████████████████████████| 890kB 38.3MB/s 
     |████████████████████████████████| 1.1MB 40.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9f8e723c84ba1c62f9deb7337731f33c4d2685807767b439811ce601586a31db
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
! pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=b7f09848f64b3b4ecdebadb346d1241a71dc32ff98121cb4c84d019cf3c9fac8
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased",do_lower_case = True)

In [ ]:
#tokenize the stories

print("story:",stories[0])
tokenized_story = tokenizer.tokenize(stories[0])
print("tokenized: ", tokenized_story)
print("token_ids: ", tokenizer.convert_tokens_to_ids(tokenized_story))

tokenized_stories = [tokenizer.tokenize(story) for story in stories]
print(tokenized_stories[0])
max_story = max(tokenized_stories,key = lambda s: len(s))
print("max_length: ",len(max_story))



story:  Rick grew up in a troubled household. He never found good support in family, and turned to gangs. It wasn't long before Rick got shot in a robbery. The incident caused him to turn a new leaf.
tokenized:  ['rick', 'grew', 'up', 'in', 'a', 'troubled', 'household', '.', 'he', 'never', 'found', 'good', 'support', 'in', 'family', ',', 'and', 'turned', 'to', 'gangs', '.', 'it', 'wasn', "'", 't', 'long', 'before', 'rick', 'got', 'shot', 'in', 'a', 'robbery', '.', 'the', 'incident', 'caused', 'him', 'to', 'turn', 'a', 'new', 'leaf', '.']
token_ids:  [6174, 3473, 2039, 1999, 1037, 11587, 4398, 1012, 2002, 2196, 2179, 2204, 2490, 1999, 2155, 1010, 1998, 2357, 2000, 18542, 1012, 2009, 2347, 1005, 1056, 2146, 2077, 6174, 2288, 2915, 1999, 1037, 13742, 1012, 1996, 5043, 3303, 2032, 2000, 2735, 1037, 2047, 7053, 1012]
['rick', 'grew', 'up', 'in', 'a', 'troubled', 'household', '.', 'he', 'never', 'found', 'good', 'support', 'in', 'family', ',', 'and', 'turned', 'to', 'gangs', '.', 'it', 'wasn

In [ ]:
from transformers import BertModel, BertConfig,BertForMultipleChoice
import torch
configuration = BertConfig()
MultiChoiceModel = BertForMultipleChoice.from_pretrained("bert-base-uncased")

In [ ]:
def get_pred(logits):
  np_logits = logits.detach().numpy()
  pred = np.argmax(np_logits)
  return pred

In [ ]:
def format_batch(stories,endings,labels):
  batch_data = []
  batch_labels = torch.LongTensor(labels)
  for i in range(len(stories)):
    batch_data.append([[stories[i],endings[i][0]],[stories[i],endings[i][1]]])
    
    print
  print(batch_data)

In [ ]:
labels = []
predictions = []

for i in range(1):
  label =torch.LongTensor([correct_endings[i],correct_endings[i+1]])
  labels.append(correct_endings[i])
  
  num_choice = 2
  batch = [[stories[i],endings[i][0]],[stories[i],endings[i][1]],[stories[i+1],endings[i+1][0]],[stories[i+1],endings[i+1][1]]]
  encoding = tokenizer.batch_encode_plus(batch,return_tensors="pt",pad_to_max_length=True)
  input_ids = encoding['input_ids']
  print(encoding.keys())
  print("input_ids_raw: ",input_ids)
  input_ids_1 = torch.stack([input_ids[0],input_ids[1]])
  input_ids_2 = torch.stack([input_ids[2],input_ids[3]])
  input_ids_total = torch.stack([input_ids_1,input_ids_2])
  print("input_ids: ", input_ids_total)

  token_type_ids = encoding['token_type_ids']
  token_type_ids_1 = torch.stack([token_type_ids[0],token_type_ids[1]])
  token_type_ids_2 = torch.stack([token_type_ids[2],token_type_ids[3]])
  token_type_total = torch.stack([token_type_ids_1,token_type_ids_2])
  print("token_types:",token_type_total)


  attention_mask = encoding['attention_mask']
  print("attention_mask:",attention_mask)
  attention_mask_1 = torch.stack([attention_mask[0],token_type_ids[1]])
  attention_mask_2 = torch.stack([attention_mask[2],attention_mask[3]])
  attention_mask_total = torch.stack([attention_mask_1,attention_mask_2])
  #print(encoding.items())
  #print({k: v.unsqueeze(0) for k,v in encoding.items()})
  
  loss,logits = MultiChoiceModel(input_ids = input_ids_total,token_type_ids = token_type_total,attention_mask = attention_mask_total, labels = label)
  print(logits)
  #print("output:",get_pred(logits))
  #predictions.append(get_pred(logits))





dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
input_ids_raw:  tensor([[  101,  6174,  3473,  2039,  1999,  1037, 11587,  4398,  1012,  2002,
          2196,  2179,  2204,  2490,  1999,  2155,  1010,  1998,  2357,  2000,
         18542,  1012,  2009,  2347,  1005,  1056,  2146,  2077,  6174,  2288,
          2915,  1999,  1037, 13742,  1012,  1996,  5043,  3303,  2032,  2000,
          2735,  1037,  2047,  7053,  1012,   102,  2002,  2003,  3407,  2085,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0],
        [  101,  6174,  3473,  2039,  1999,  1037, 11587,  4398,  1012,  2002,
          2196,  2179,  2204,  2490,  1999,  2155,  1010,  1998,  2357,  2000,
         18542,  1012,  2009,  2347,  1005,  1056,  2146,  2077,  6174,  2288,
          2915,  1999,  1037, 13742,  1012,  1996,  5043,  3303,  2032,  2000,
          2735,  1037,  2047,  7053,  1012,   102,  2002,  2587,  1037,  6

In [ ]:
""

In [ ]:
#calculate the accuracy
correct =0
for i in range(len(labels)):
  print("label:",labels[i])
  print("pred:",predictions[i])
  if predictions[i] == labels[i]:
    correct +=1
print("accuracy:",correct/len(labels))

label: 0
pred: 1
label: 0
pred: 0
label: 1
pred: 0
label: 0
pred: 1
label: 0
pred: 0
label: 0
pred: 1
label: 1
pred: 1
label: 0
pred: 0
label: 1
pred: 1
label: 0
pred: 0
label: 0
pred: 0
label: 1
pred: 1
label: 1
pred: 1
label: 1
pred: 0
label: 0
pred: 0
label: 1
pred: 1
label: 1
pred: 0
label: 1
pred: 1
label: 0
pred: 1
label: 0
pred: 0
accuracy: 0.65
